In [ ]:
from  lxml import etree
from os import listdir,stat
from os.path import join

In [ ]:
file_dir=r'C:\Users\Kart\jupyter_projects\web\broker_info'

In [ ]:
class open_broker_report:
    __slots__ ='file_path','root'
    def __init__(self,_file_dir):  
        self.file_path=self.__get_last_report(_file_dir)
        self.root=self.__load_xml(self.file_path).getroot()
    
    def __get_last_report(self,folder_name:str):
        reports_list=[join(file_dir,_file) for _file in listdir(folder_name) if _file[-3:]=='xml']
        reports_list=[]
        for _file in listdir(file_dir):
            if _file[-3:]=='xml':
                _file_path=join(file_dir,_file)
                reports_list.append((_file_path,stat(_file_path).st_mtime))
        return sorted(reports_list, key=lambda tup: tup[1])[-1][0]  
    
    def __load_xml(self,file_path:str):
        #parser = etree.XMLParser(encoding='cp1251')
        return etree.parse(file_path)
    
    def __children_to_dict(self,node_name:str,
                           d_keys:list=None,
                           print_uniq_keys:bool=False,
                          float_keys:list=[]):
        if print_uniq_keys:
            d_keys=None
            
        if d_keys is not None:  
            res={}
        else:
            res=[]
            
        for _el in self.root.find(node_name).getchildren():
            _d_el=dict(_el.items())
            _d_el_keys=_d_el.keys()
            for _f_key in float_keys:
                if _f_key in _d_el_keys:
                    _d_el[_f_key]=float(_d_el[_f_key])
            if d_keys is not None:
                _pop_items=[]
                for _k in d_keys[::-1]:
                    _p=_d_el.pop(_k) 
                    _pop_items.append(_p)
                _final_d={_pop_items[0]:_d_el}
                for _k in _pop_items[1:]:
                    _final_d={_k:_final_d}
                    
                upd_res=res
                for _p in _pop_items[::-1]:
                    if _p in  upd_res.keys():
                        upd_res=upd_res[_p]
                        _final_d=_final_d[_p]
                    else:
                        upd_res[_p]=_final_d[_p]
                        #print(res)
                        break
               
                #res.update(_final_d)
            else:
                res.append(_d_el) 
            
        if print_uniq_keys:
            return set().union(*[list(_el.keys()) for _el in res])
        
        return res
    
    def json_report(self):
        pass
    
    def _extract_title_info(self,print_raw:bool=False)->dict:
        title_dict=dict(self.root.items())
        if print_raw:
            return title_dict
        
        res={'board_list':list(map(str.strip,title_dict['board_list'].split(','))),
            'period':(title_dict['date_from'],title_dict['date_to']),
             'cliend_id':{'client_code':title_dict['client_code'],
                          'micex_trade_code':title_dict['micex_trade_code']
                         },
             'agreement':{'agreement_number':title_dict['agreement_number'],
                          'agreement_date':title_dict['agreement_date']
                         } ,
             'report_date':title_dict['build_date']
            }
        return res
    
    def _extract_section_definition(self,print_raw:bool=False,hide_empty:bool=False):
        res={}
        for _el in self.root.find('section_definition').getchildren():
            _d_el=dict(_el.items())
            if hide_empty:
                if not(list(_d_el.keys())==['name'] or _d_el=={}):
                    res[_el.tag]=_d_el
            else:
                res[_el.tag]=_d_el
                    
        if print_raw:
            return res
        return "no code written"
    
    def _extract_unified_account_totally(self,print_raw:bool=False):  
        if print_raw:
            d_keys,float_keys=None,[]
        else:
            d_keys,float_keys=['row_name','currency'],['value']
        _d=self.__children_to_dict('unified_account_totally',d_keys=d_keys,float_keys=float_keys)
        if print_raw:
            return _d
        _mapping={'Входящий остаток (факт)':'income_balance_fact',
                 'Вознаграждение Брокера за организацию доступа к биржевым торгам с предоставлением информации клиентам, необходимой для совершения операций и сделок':'broker_fee_for_info',
                 'Движение средств':'cashflow',
                 'Комиссия Брокера/Доп. комиссия Брокера "Сборы ТС" за заключение сделок':'broker_fee_for_deals',
                 'НКД':'ncd',
                 'Погашение НКД':'repayment_ncd',
                 'Погашение ЦБ':'repayment_cb',
                 'Сальдо расчётов по сделкам с ценными бумагами':'securities_deals_saldo',
                 'Исходящий остаток (факт)':'outcome_balance_fact',
                 'Оценка активов, по курсу ЦБ (факт)':'assets_cb_value_fact',
                 'Оценка активов, по курсу ЦБ (факт+план) (с учётом нерассчитанных сделок по ценным бумагам)':'assets_cb_value_fact_plan',
                 'Оценка активов, по Центральному курсу (факт) (Предварительное закрытие дня)':'assets_cb_value_fact_precloseday',
                 'Оценка активов, по Центральному курсу (факт+план) (с учётом нерассчитанных сделок по ценным бумагам)  (Предварительное закрытие дня)':'assets_cb_value_fact_plan_precloseday',
                 'Оценка активов, по Центральному курсу (факт) (Итоговое закрытие дня)':'assets_cb_value_fact_finalcloseday',
                 'Оценка активов, по Центральному курсу (факт+план) (с учётом нерассчитанных сделок по ценным бумагам)  (Итоговое закрытие дня)':'assets_cb_value_fact_plan_finalcloseday'
                 }
        return {_mapping[_key]:_item for _key,_item in _d.items()}
    
    def _extract_unified_guarantee_totally(self,print_raw:bool=False): 
        if print_raw:
            d_keys,float_keys=None,[]
        else:
            d_keys,float_keys=['active_go'],['amount','amount_plan','course','collateral','highly_liquid']

        _d=self.__children_to_dict('unified_guarantee_totally',float_keys=float_keys,d_keys=d_keys)
        if print_raw:
            return _d
        return _d
    
    def _extract_unified_preclosing_assets(self,print_raw:bool=False): 
        if print_raw:
            d_keys,float_keys=None,[]
        else:
            d_keys,float_keys=['asset_name'],['closing_position_fact',
                                             'closing_position_plan',
                                             'blocked_for_out',
                                             'settlement_price',
                                             'currency_course',
                                             'settlement_fact',
                                             'settlement_plan']

        _d=self.__children_to_dict('unified_preclosing_assets',float_keys=float_keys,d_keys=d_keys)
        if print_raw:
            return _d
        return _d   
    
    def _extract_unified_closing_assets(self,print_raw:bool=False): 
        if print_raw:
            d_keys,float_keys=None,[]
        else:
            d_keys,float_keys=['asset_name'],['opening_position_fact',
                                             'opening_position_plan',
                                             'period_change_position_fact',
                                             'period_change_position_plan',
                                             'closing_position_fact',
                                             'closing_position_plan',
                                             'blocked_for_out',
                                              'settlement_price',
                                              'currency_course_cb',
                                              'settlement_fact_cb',
                                              'settlement_plan_cb',
                                              'currency_course_ck',
                                              'settlement_fact_ck',
                                              'settlement_plan_ck',
                                              'position_weight_cb',
                                              'position_weight_ck'
                                             ]

        _d=self.__children_to_dict('unified_closing_assets',float_keys=float_keys,d_keys=d_keys)
        if print_raw:
            return _d
        return _d   
    
    def _extract_spot_main_deals_conclusion(self,print_raw:bool=False): 
        if print_raw:
            float_keys=[]
        else:
            float_keys=['buy_qnty',
                         'price',
                         'volume_currency',
                         'volume_rur',
                         'broker_commission'
                       ]

        _d=self.__children_to_dict('spot_main_deals_conclusion',float_keys=float_keys,d_keys=None)
        if print_raw:
            return _d
        return _d     
    def _extract_spot_main_deals_executed(self,print_raw:bool=False):       
        if print_raw:
            float_keys=[]
        else:
            float_keys=['buy_qnty',
                         'price',
                         'volume_currency_price',
                         'price_currency_rate',
                         'volume'
                       ]

        _d=self.__children_to_dict('spot_main_deals_executed',float_keys=float_keys,d_keys=None)
        if print_raw:
            return _d
        return _d           
    
    def _extract_unified_non_trade_money_operations(self,print_raw:bool=False): 
        if print_raw:
            float_keys=[]
        else:
            float_keys=['amount']
        _d=self.__children_to_dict('unified_non_trade_money_operations',float_keys=float_keys,d_keys=None)
        if print_raw:
            return _d
        return _d 
    
    def _extract_unified_spot_non_trade_security_operations(self,print_raw:bool=False): 
        if print_raw:
            float_keys=[]
        else:
            float_keys=['quantity']
        _d=self.__children_to_dict('spot_non_trade_security_operations',float_keys=float_keys,d_keys=None)
        if print_raw:
            return _d
        return _d   
    
    def _extract_spot_portfolio_security_params(self,print_raw:bool=False): 
        if print_raw:
            d_keys,float_keys=None,[]
        else:
            d_keys,float_keys=['security_name'],['nominal',
                                                 'lot_size',
                                                 'market_price'
                                                ]

        _d=self.__children_to_dict('spot_portfolio_security_params',float_keys=float_keys,d_keys=d_keys)
        if print_raw:
            return _d
        return _d 



In [ ]:
ob=open_broker_report(file_dir)
#ob._extract_title_info()
#ob._extract_section_definition(True,True) 
#ob._extract_unified_account_totally()
#ob._extract_unified_guarantee_totally()

#ob._extract_unified_preclosing_assets()
#ob._extract_unified_closing_assets()
#ob._extract_spot_main_deals_conclusion()
#ob._extract_spot_main_deals_executed()
#ob._extract_unified_non_trade_money_operations()

#ob._extract_unified_spot_non_trade_security_operations()
ob._extract_spot_portfolio_security_params()

In [ ]:
#ob._extract_title_info(print_raw=True)
#ob._extract_section_definition(True,False) 
#ob._extract_unified_account_totally(print_raw=True)
#ob._extract_unified_guarantee_totally(print_raw=True)

#ob._extract_unified_preclosing_assets(print_raw=True)
#ob._extract_unified_closing_assets(print_raw=True)
#ob._extract_spot_main_deals_conclusion(print_raw=True)
#ob._extract_spot_main_deals_executed(print_raw=True)
#ob._extract_unified_non_trade_money_operations(print_raw=True)

#ob._extract_unified_spot_non_trade_security_operations(print_raw=True)
ob._extract_spot_portfolio_security_params(print_raw=True)